In [1]:
import os
import pandas as pd

In [2]:
RESULTS_DIR = "../../data/chen_results/result/"

In [37]:
all_dirs = os.listdir(RESULTS_DIR)

In [38]:
all_metric_files = {}
for result_dir in all_dirs:
    path = os.path.join(RESULTS_DIR, result_dir)
    metric_dir = os.path.join(path, "metric")
    
    metric_files = [filename for filename in os.listdir(metric_dir) if "metrics" in filename]
    
    all_metric_files[metric_dir] = metric_files

In [39]:
results_by_data = {}
results_by_model = {}
max_f1 = 0
best_by_f1 = None
max_precision = 0
best_by_precision = None
max_recall = 0
best_by_recall = None
max_auc = 0
best_by_auc = None
for metric_dir, filenames in all_metric_files.items():
    for filename in filenames:
        data = os.path.split(os.path.split(metric_dir)[0])[1]
        model = filename.replace(data, "")
        
        metrics = pd.read_csv(os.path.join(metric_dir, filename))
        mean_recall = metrics.iloc[0]["mean_test_recall"]
        if mean_recall > max_recall:
            max_recall = mean_recall
            best_by_recall = (data, model)
            
        mean_precision = metrics.iloc[0]["mean_test_precision"]
        if mean_precision > max_precision:
            max_precision = mean_precision
            best_by_precision = (data, model)
            
        mean_average_precision = metrics.iloc[0]["mean_test_average_precision"]
        
        mean_f1 = metrics.iloc[0]["mean_test_f1"]
        if mean_f1 > max_f1:
            max_f1 = mean_f1
            best_by_f1 = (data, model)
            
        mean_roc_auc = metrics.iloc[0]["mean_test_roc_auc"]
        if mean_roc_auc > max_auc:
            max_auc = mean_roc_auc
            best_by_auc = (data, model)
            
        metric_dict = {
            "f1": mean_f1,
            "precision": mean_precision,
            "recall": mean_recall,
            "avg_precision": mean_average_precision,
            "roc_auc": mean_roc_auc
        }
        
        
        results_by_data[data] = results_by_data.get(data, {})
        results_by_data[data][model] = metric_dict
        
        results_by_model[model] = results_by_model.get(model, {})
        results_by_model[model][data] = metric_dict

In [40]:
print(f"Max F1: {max_f1}")
print(f"Best by F1: {best_by_f1}")
print("\n")
print(f"Max precision: {max_precision}")
print(f"Best by precision: {best_by_precision}")
print("\n")
print(f"Max recall: {max_recall}")
print(f"Best by recall: {best_by_recall}")
print("\n")
print(f"Max ROC AUC: {max_auc}")
print(f"Best by ROC AUC: {best_by_auc}")

Max F1: 0.6296561127854439
Best by F1: ('feature_pc_avg_pad', '_MLP_cv_metrics.csv')


Max precision: 1.0
Best by precision: ('feature_embedding_small_4_6', '_RF_cv_metrics.csv')


Max recall: 0.9922402159244263
Best by recall: ('feature_pc_stats_PP', '_GaussianBayes_cv_metrics.csv')


Max ROC AUC: 0.8752052234692302
Best by ROC AUC: ('feature_pc_avg_pad', '_RF_cv_metrics.csv')


In [42]:
def get_avg_in_group(metric, group_dict):
    avg_metric = 0
    count = 0
    for subgroup, metric_dict in group_dict.items():
        avg_metric += metric_dict[metric]
        count += 1
    return avg_metric / count

In [45]:
for data_type, res_dict in results_by_data.items():
    avg_f1 = get_avg_in_group("f1", res_dict)
    if avg_f1 > 0.35:
        print(f"Average F1 for {data_type}: {avg_f1}")

Average F1 for feature_embedding_original_2_1: 0.354523494244844
Average F1 for feature_embedding_original_3_3: 0.36963000715816186
Average F1 for feature_embedding_original_3_5: 0.3883951965458012
Average F1 for feature_embedding_original_3_7: 0.3684882645200857
Average F1 for feature_embedding_original_4_1: 0.37037290863376354
Average F1 for feature_embedding_random_2_4: 0.3834857135787189
Average F1 for feature_embedding_random_2_6: 0.3798027216477011
Average F1 for feature_embedding_random_3_6: 0.354328975339283
Average F1 for feature_embedding_scrambled_2_5: 0.35547887291021774
Average F1 for feature_embedding_scrambled_2_7: 0.3902900329033624
Average F1 for feature_embedding_scrambled_3_1: 0.36348538786341567
Average F1 for feature_embedding_scrambled_3_3: 0.3931111919030269
Average F1 for feature_embedding_scrambled_3_4: 0.3584356328114424
Average F1 for feature_embedding_scrambled_3_5: 0.40498354399316483
Average F1 for feature_embedding_scrambled_3_6: 0.35982441397134324
Avera

In [48]:
for data_type, res_dict in results_by_data.items():
    avg_auc = get_avg_in_group("roc_auc", res_dict)
    if avg_auc > 0.75:
        print(f"Average ROC AUC for {data_type}: {avg_auc}")

Average ROC AUC for feature_embedding_original_2_2: 0.7797475579033304
Average ROC AUC for feature_embedding_original_2_3: 0.7615816858580016
Average ROC AUC for feature_embedding_original_2_4: 0.7661240450268463
Average ROC AUC for feature_embedding_original_3_1: 0.778376077268798
Average ROC AUC for feature_embedding_original_3_3: 0.7618981008380413
Average ROC AUC for feature_embedding_original_3_5: 0.7672898838216115
Average ROC AUC for feature_embedding_original_3_6: 0.7838134523936732
Average ROC AUC for feature_embedding_original_3_7: 0.7532543029975117
Average ROC AUC for feature_embedding_original_4_1: 0.7709255354506841
Average ROC AUC for feature_embedding_original_4_3: 0.7706821646449193
Average ROC AUC for feature_embedding_random_2_3: 0.7562619947610397
Average ROC AUC for feature_embedding_random_2_4: 0.7609144253101172
Average ROC AUC for feature_embedding_random_2_5: 0.7575209139332654
Average ROC AUC for feature_embedding_random_3_2: 0.7538121997555359
Average ROC AUC

In [51]:
for model_type, res_dict in results_by_model.items():
    avg_f1 = get_avg_in_group("f1", res_dict)
    print(f"Average F1 for {model_type}: {avg_f1}")

Average F1 for _Dummy_cv_metrics.csv: 0.1988930776520885
Average F1 for _GaussianBayes_cv_metrics.csv: 0.39700801389548845
Average F1 for _KNN_cv_metrics.csv: 0.39557178496017803
Average F1 for _LogiReg_cv_metrics.csv: 0.2432919623959905
Average F1 for _MLP_cv_metrics.csv: 0.48249829460746296
Average F1 for _RF_cv_metrics.csv: 0.18576144599082497
Average F1 for _SVM_cv_metrics.csv: 0.15057548409669416


In [52]:
for model_type, res_dict in results_by_model.items():
    avg_auc = get_avg_in_group("roc_auc", res_dict)
    print(f"Average ROC AUC for {model_type}: {avg_auc}")

Average ROC AUC for _Dummy_cv_metrics.csv: 0.49870001997735924
Average ROC AUC for _GaussianBayes_cv_metrics.csv: 0.6916717792252095
Average ROC AUC for _KNN_cv_metrics.csv: 0.7292887746503045
Average ROC AUC for _LogiReg_cv_metrics.csv: 0.6927223535293048
Average ROC AUC for _MLP_cv_metrics.csv: 0.7724637202683242
Average ROC AUC for _RF_cv_metrics.csv: 0.7386778976467385
Average ROC AUC for _SVM_cv_metrics.csv: 0.6976198553250885
Average ROC AUC for _XGBoost_cv_metrics.csv: nan


In [53]:
for model_type, res_dict in results_by_model.items():
    avg_pr = get_avg_in_group("precision", res_dict)
    print(f"Average precision for {model_type}: {avg_pr}")

Average precision for _Dummy_cv_metrics.csv: 0.20001772036721172
Average precision for _GaussianBayes_cv_metrics.csv: 0.3481138737841311
Average precision for _KNN_cv_metrics.csv: 0.49866702947251906
Average precision for _LogiReg_cv_metrics.csv: 0.41309415178201137
Average precision for _MLP_cv_metrics.csv: 0.559515386801247
Average precision for _RF_cv_metrics.csv: 0.6202845965123635
Average precision for _SVM_cv_metrics.csv: 0.26916143566042383
Average precision for _XGBoost_cv_metrics.csv: nan


In [54]:
for model_type, res_dict in results_by_model.items():
    avg_rec = get_avg_in_group("recall", res_dict)
    print(f"Average recall for {model_type}: {avg_rec}")

Average recall for _Dummy_cv_metrics.csv: 0.19980506822612074
Average recall for _GaussianBayes_cv_metrics.csv: 0.50261482010708
Average recall for _KNN_cv_metrics.csv: 0.33897003695765293
Average recall for _LogiReg_cv_metrics.csv: 0.18833628818148937
Average recall for _MLP_cv_metrics.csv: 0.42949864606211363
Average recall for _RF_cv_metrics.csv: 0.11956859216921135
Average recall for _SVM_cv_metrics.csv: 0.12488423169537717
Average recall for _XGBoost_cv_metrics.csv: nan


In [56]:
import glob

In [58]:
f1 = {'ranking':'rank_test_f1', 'score':'mean_test_f1', 'metric':'f1'}
ranking = f1["ranking"]
best_metrics_all = pd.DataFrame(columns=[])
for root, dirs, _ in os.walk(RESULTS_DIR, topdown=False):
    for name in dirs:
        files_in_dir = glob.glob(os.path.join(root, name, '*cv_metrics.csv'))
        for metric_file in files_in_dir:
            if 'small' in metric_file or 'scrambled' in metric_file or 'uniform' in metric_file or 'python' in metric_file: continue
            # if not ('original_3' in metric_file or 'original_1' in metric_file): continue
            metric = pd.read_csv(metric_file)
            metric = metric.loc[metric[ranking]==1].head(1)
            descriptor = os.path.split(metric_file)[-1].split('.')[0]
            is_embedding = "embedding" in descriptor
            metric['descriptor'] = descriptor
            metric['is_embedding'] = is_embedding
            if best_metrics_all.empty:
                best_metrics_all = metric
            else:
                best_metrics_all = best_metrics_all.append(metric)
best_metrics_all = best_metrics_all.sort_values(by=['is_embedding'], ascending=True)


In [59]:
best_metrics_all

,Unnamed: 0,mean_fit_time,std_fit_time,mean_score_time,std_score_time,params,split0_test_recall,split1_test_recall,split2_test_recall,split3_test_recall,...,param_RF__max_features,param_RF__max_depth,param_SVM__C,param_SVM__gamma,param_SVM__kernel,param_XGBoost__learning_rate,param_XGBoost__loss,param_XGBoost__max_depth,param_XGBoost__max_features,param_XGBoost__n_estimators
1,1,4.883232,0.084471,0.012497,0.006249,{'XGBoost__learning_rate': 0.03840502122219712...,0.410256,0.487179,0.410256,0.435897,...,NaN,NaN,NaN,NaN,NaN,0.038405,exponential,29.0,27.0,191.0
0,0,0.151370,0.023855,0.020149,0.005721,{},0.333333,0.282051,0.256410,0.179487,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,0.211538,0.044880,0.031616,0.008176,{},0.743590,0.564103,0.641026,0.820513,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,0.530552,0.273188,0.974795,0.408257,{'KNN__n_neighbors': 5},0.512821,0.487179,0.512821,0.461538,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,29,2.433096,0.021650,0.013065,0.000941,"{'LogiReg__C': 0.46469662500627235, 'LogiReg__...",0.589744,0.564103,0.512821,0.487179,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10,10,1.922988,0.034544,0.053112,0.007654,"{'SVM__C': 0.4700668928852339, 'SVM__gamma': 0...",0.333333,0.384615,0.410256,0.282051,...,NaN,NaN,0.470067,0.017025,linear,NaN,NaN,NaN,NaN,NaN
19,19,0.956026,0.009373,0.003124,0.006249,{'XGBoost__learning_rate': 0.05841334696737744...,0.179487,0.153846,0.153846,0.128205,...,NaN,NaN,NaN,NaN,NaN,0.058413,deviance,5.0,25.0,61.0
0,0,0.015611,0.000015,0.004680,0.007149,{},0.153846,0.307692,0.282051,0.179487,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,0,0.015622,0.006985,0.007811,0.007811,{},0.256410,0.153846,0.179487,0.179487,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
best_metrics_all.columns

Index(['Unnamed: 0', 'mean_fit_time', 'std_fit_time', 'mean_score_time',
       'std_score_time', 'params', 'split0_test_recall', 'split1_test_recall',
       'split2_test_recall', 'split3_test_recall',
       ...
       'param_RF__max_features', 'param_RF__max_depth', 'param_SVM__C',
       'param_SVM__gamma', 'param_SVM__kernel', 'param_XGBoost__learning_rate',
       'param_XGBoost__loss', 'param_XGBoost__max_depth',
       'param_XGBoost__max_features', 'param_XGBoost__n_estimators'],
      dtype='object', length=148)

In [61]:
def get_best_in_group(metric, group_dict):
    metrics = []
    for subgroup, metric_dict in group_dict.items():
        metrics.append(metric_dict[metric])
    return max(metrics)

In [66]:
for data_type, res_dict in results_by_data.items():
    avg_f1 = get_best_in_group("f1", res_dict)
    if avg_f1 > 0.6:
        print(f"Average F1 for {data_type}: {avg_f1}")

Average F1 for feature_embedding_original_3_2: 0.614753231449919
Average F1 for feature_embedding_original_3_4: 0.6079159508309917
Average F1 for feature_embedding_original_3_5: 0.6138045247685476
Average F1 for feature_embedding_original_3_7: 0.6028489071278202
Average F1 for feature_embedding_scrambled_3_4: 0.6011541884277452
Average F1 for feature_embedding_scrambled_3_6: 0.6020322835233985
Average F1 for feature_embedding_scrambled_4_3: 0.6087101680751047
Average F1 for feature_pc_avg_pad: 0.6296561127854439
Average F1 for feature_pc_train_zeropad: 0.6251482719378803
Average F1 for feature_protparam: 0.6202016895341236


In [68]:
for data_type, res_dict in results_by_model.items():
    avg_f1 = get_best_in_group("f1", res_dict)
    print(f"Average F1 for {data_type}: {avg_f1}")

Average F1 for _Dummy_cv_metrics.csv: 0.2460750959666272
Average F1 for _GaussianBayes_cv_metrics.csv: 0.5004031532494713
Average F1 for _KNN_cv_metrics.csv: 0.5332066465972185
Average F1 for _LogiReg_cv_metrics.csv: 0.5408318870459398
Average F1 for _MLP_cv_metrics.csv: 0.6296561127854439
Average F1 for _RF_cv_metrics.csv: 0.5599735058516643
Average F1 for _SVM_cv_metrics.csv: 0.5943359094218925
Average F1 for _XGBoost_cv_metrics.csv: 0.5269460841888727
